# **PROJET DE MASTER 1**

## **OBjectif principal: Analyse de données de criminalité; étude de cas sur les données de la base Crime30k.csv**

#### Période: Décembre 2025

# **1. Introduction et problématique**



La sécurité publique constitue un enjeu central pour les grandes villes, où les forces de l'ordre doivent faire face à une criminalité à la fois diffuse dans l'espace et variable dans le temps. Dans ce contexte, disposer d'outils d'analyse permettant de transformer les données de criminalité en décisions opérationnelles devient essentiel pour orienter les patrouilles, cibler les actions de prévention et affecter efficacement les effectifs disponibles.

La base de données Crime30.csv, qui recense les incidents criminels enregistrés dans la ville de Chicago sur une période donnée, offre un support riche pour ce travail. Chaque incident y est décrit par le type d'infraction, les circonstances (lieu, heure, description, contexte domestique), l'issue policière (arrestation ou non), ainsi que par des informations temporelles et spatiales fines (date, heure, quartier, coordonnées). Cette structure permet à la fois une lecture globale des niveaux de criminalité et une analyse plus ciblée des profils d'infractions selon le temps, l'espace et la réponse policière.

Dans ce projet, l'objectif n'est donc pas seulement de “décrire” la criminalité, mais de l'analyser dans une perspective d'aide à la décision : identifier les zones et créneaux horaires les plus exposés, repérer les types d'infractions les plus consommateurs de ressources, et proposer des critères concrets pour allouer de nouveaux agents sur le territoire.




Notre problématique consiste alors à identifier les profils spatio-temporels d'infractions et orienter de manière optimale l'allocation des effectifs policiers (en termes de lieux, de périodes et de compétences).

Afin de résoudre cette problématique, nous allons à travers cette étude, essayer de répondre aux questions suivantes:

* *Quels profils de criminalité urbaine observe-t-on dans la zone étudiée, en combinant l'espace (beats, districts, quartiers), le temps (heure, jour, saison, année), le type d'infraction, le contexte domestique et l'issue policière (arrestation ou non)?*

* *Quels facteurs (type d'infraction, localisation, créneau horaire, contexte domestique, durée de traitement) sont significativement associés à la probabilité d'arrestation et à la charge opérationnelle des services de police?*

* *Comment les types de criminalité et la pression opérationnelle évoluent-ils selon les quartiers et la temporalité (jours, saisons, années), et comment ces dynamiques peuvent-elles être traduites en priorités d'affectation pour les nouveaux agents?*

# **2. Exploration et préparation des données**

In [2]:
# Importattion des libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
import warnings
from matplotlib.ticker import MaxNLocator

# Chargement des données
data = pd.read_csv('crimes30k.csv')


In [3]:
data.head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11021273,JA351713,07/10/2017 09:50:00 AM,002XX N MICHIGAN AVE,0820,THEFT,$500 AND UNDER,COMMERCIAL / BUSINESS OFFICE,False,False,...,42.0,32.0,06,NaN,NaN,2017,07/18/2017 03:52:43 PM,NaN,NaN,NaN
1,10928464,JA242180,07/30/2012 11:05:00 AM,027XX W 84TH PL,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,NaN,False,False,...,18.0,70.0,11,NaN,NaN,2012,04/29/2017 03:49:38 PM,NaN,NaN,NaN
2,10122816,HY311916,12/10/2008 12:00:00 PM,085XX S ESCANABA AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,RESIDENCE,False,False,...,10.0,46.0,11,NaN,NaN,2008,08/17/2015 03:03:40 PM,NaN,NaN,NaN
3,1448084,G177358,03/28/2001 10:00:00 PM,005XX E 33 ST,5001,OTHER OFFENSE,OTHER CRIME INVOLVING PROPERTY,PARKING LOT/GARAGE(NON.RESID.),False,False,...,NaN,NaN,26,NaN,NaN,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN
4,1793881,G617937,10/14/2001 03:50:00 PM,062XX N MC CORMICK RD,0460,BATTERY,SIMPLE,PARKING LOT/GARAGE(NON.RESID.),True,False,...,NaN,NaN,08B,NaN,NaN,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN


In [4]:
data.shape

(30000, 22)

Cette base de données contient 22 variables qui fournissent des informations sur l'enregistrement des actes d'incidents répertoriés et 30 000 lignes représentant des actes enrégistrés. Nous allons devoir visualiser les premières et dernières lignes de la base afin de se faire une idée rapide des différentes données auxquelles nous avons affaire.

In [5]:
# Information sur le type des données de la base
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    30000 non-null  int64  
 1   Case Number           30000 non-null  object 
 2   Date                  30000 non-null  object 
 3   Block                 30000 non-null  object 
 4   IUCR                  30000 non-null  object 
 5   Primary Type          30000 non-null  object 
 6   Description           30000 non-null  object 
 7   Location Description  29955 non-null  object 
 8   Arrest                30000 non-null  bool   
 9   Domestic              30000 non-null  bool   
 10  Beat                  30000 non-null  int64  
 11  District              30000 non-null  int64  
 12  Ward                  27413 non-null  float64
 13  Community Area        27420 non-null  float64
 14  FBI Code              30000 non-null  object 
 15  X Coordinate       


**Type entier (4) :** ID, Beat, District, Year

**Type chaîne de caractères (10) :** Case Number, Date, Block, IUCR, Primary Type, Description, Location Description, Updated On, FBI Code, Location

**Type Boolean (2) :** Arrest, Domestic

**Type réel (6) :** Ward, Community Area, Latitude, Longitude, X Coordinate, Y Coordinate

### Conversion de la colonne "Date"

In [6]:
data["Date"] = pd.to_datetime(data["Date"], format="%m/%d/%Y %I:%M:%S %p")

#Création des variables temporelles
data["year"]    = data["Date"].dt.year
data["month"]   = data["Date"].dt.month
data["day"]     = data["Date"].dt.day
data["hour"]    = data["Date"].dt.hour
data["weekday"] = data["Date"].dt.day_name()

In [7]:
# Visualisation des premières lignes
data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Year,Updated On,Latitude,Longitude,Location,year,month,day,hour,weekday
0,11021273,JA351713,2017-07-10 09:50:00,002XX N MICHIGAN AVE,0820,THEFT,$500 AND UNDER,COMMERCIAL / BUSINESS OFFICE,False,False,...,2017,07/18/2017 03:52:43 PM,NaN,NaN,NaN,2017,7,10,9,Monday
1,10928464,JA242180,2012-07-30 11:05:00,027XX W 84TH PL,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,NaN,False,False,...,2012,04/29/2017 03:49:38 PM,NaN,NaN,NaN,2012,7,30,11,Monday
2,10122816,HY311916,2008-12-10 12:00:00,085XX S ESCANABA AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,RESIDENCE,False,False,...,2008,08/17/2015 03:03:40 PM,NaN,NaN,NaN,2008,12,10,12,Wednesday
3,1448084,G177358,2001-03-28 22:00:00,005XX E 33 ST,5001,OTHER OFFENSE,OTHER CRIME INVOLVING PROPERTY,PARKING LOT/GARAGE(NON.RESID.),False,False,...,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN,2001,3,28,22,Wednesday
4,1793881,G617937,2001-10-14 15:50:00,062XX N MC CORMICK RD,0460,BATTERY,SIMPLE,PARKING LOT/GARAGE(NON.RESID.),True,False,...,2001,08/17/2015 03:03:40 PM,NaN,NaN,NaN,2001,10,14,15,Sunday


Il est clair de constater la présence de quelques valeurs manquantes (NaN) à partir de la variable "Location Description" jusqu'à la dernière.

In [8]:
# Visualisation des dernières lignes
data.tail()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Year,Updated On,Latitude,Longitude,Location,year,month,day,hour,weekday
29995,9995418,HY186092,2015-03-15 23:07:00,059XX S CAMPBELL AVE,041A,BATTERY,AGGRAVATED: HANDGUN,RESIDENCE PORCH/HALLWAY,False,False,...,2015,02/10/2018 03:50:01 PM,41.785660,-87.686257,"(41.785659563, -87.686257097)",2015,3,15,23,Sunday
29996,9998420,HY188618,2015-03-17 18:00:00,068XX S EAST END AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,2015,02/10/2018 03:50:01 PM,41.770648,-87.583787,"(41.770648384, -87.583786515)",2015,3,17,18,Tuesday
29997,9998516,HY188592,2014-11-19 10:14:00,011XX W DIVISION ST,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,OTHER,False,True,...,2014,02/10/2018 03:50:01 PM,41.903527,-87.656041,"(41.903527055, -87.656040878)",2014,11,19,10,Wednesday
29998,9999014,HY189066,2015-03-17 19:00:00,003XX E WACKER DR,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,2015,02/10/2018 03:50:01 PM,41.887804,-87.618633,"(41.887803629, -87.618633342)",2015,3,17,19,Tuesday
29999,9999860,HY189824,2015-03-11 12:30:00,004XX S HAMLIN BLVD,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,2015,02/10/2018 03:50:01 PM,41.875058,-87.720448,"(41.875057848, -87.720447562)",2015,3,11,12,Wednesday


En visualisant des dernières lignes, le constat que nous faisons semble être différent au niveau des données car pas une seule valeur manquante est observée dans les séries colonnes de la base. Il sera alors très utile de comptabiliser l'ensemble des valeurs manquantes afin d'effectuer un nettoyage important.

## A. Nettoyage et préparation des données

In [9]:
# Gestion des valeurs manquantes.
data.isna().sum()

ID                         0
Case Number                0
Date                       0
Block                      0
IUCR                       0
Primary Type               0
Description                0
Location Description      45
Arrest                     0
Domestic                   0
Beat                       0
District                   0
Ward                    2587
Community Area          2580
FBI Code                   0
X Coordinate             315
Y Coordinate             315
Year                       0
Updated On                 0
Latitude                 315
Longitude                315
Location                 315
year                       0
month                      0
day                        0
hour                       0
weekday                    0
dtype: int64

Sur les 22 variables, 7 contiennent un nombre important de valeurs manquantes qui pourraient compliquer la manipulation des données dans le cadre de notre analyse. Nous observons 45 valeurs manquantes pour la variable "Location Description", 2 587 pour "Ward", 2 580 pour "Community Area", 315 pour "X Coordinate" et "Y Coordinate", ainsi que 315 pour "Latitude", "Longitude" et "Location". Étant donné le type de ces variables, nous pourrions soit supprimer toutes les lignes contenant des valeurs manquantes, soit remplacer toutes ces valeurs manquantes par des zéros ou par la moyenne de chacune des variables de type entier ou réel. Cette seconde option pourrait avoir un impact très significatif sur les données, car certaines variables sont de type "booléen" et ne pourront pas être remplacées par des zéros. La solution idéale serait donc de nous débarrasser de toutes les valeurs manquantes, même si cela pourrait entraîner la perte d'informations.

In [10]:
# Suppression des valeurs manquantes
data.dropna(inplace=True)
data

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Year,Updated On,Latitude,Longitude,Location,year,month,day,hour,weekday
291,11290222,JB230424,2018-04-19 20:55:00,039XX N FREMONT ST,0265,CRIMINAL SEXUAL ASSAULT,AGGRAVATED - OTHER,RESIDENCE - PORCH / HALLWAY,True,False,...,2018,06/17/2020 03:54:43 PM,41.954017,-87.652106,"(41.954016661, -87.652105978)",2018,4,19,20,Thursday
292,24580,JC287360,2019-06-01 06:35:00,043XX N MOZART ST,0110,HOMICIDE,FIRST DEGREE MURDER,AUTO,False,False,...,2019,06/20/2020 03:48:45 PM,41.960145,-87.699654,"(41.960145088, -87.699653761)",2019,6,1,6,Saturday
295,11711231,JC292289,2019-06-02 17:00:00,053XX N SHERIDAN RD,0620,BURGLARY,UNLAWFUL ENTRY,APARTMENT,False,False,...,2019,06/30/2019 03:56:27 PM,41.979122,-87.655057,"(41.97912164, -87.655056515)",2019,6,2,17,Sunday
296,11714958,JC297015,2019-06-08 00:20:00,015XX S CALIFORNIA BLVD,2093,NARCOTICS,FOUND SUSPECT NARCOTICS,HOSPITAL BUILDING/GROUNDS,True,False,...,2019,06/30/2019 03:56:27 PM,41.860911,-87.695745,"(41.860911204, -87.695744714)",2019,6,8,0,Saturday
297,10016840,HY206110,2015-04-01 09:45:00,058XX W BYRON ST,0810,THEFT,OVER $500,STREET,False,False,...,2015,02/10/2018 03:50:01 PM,41.951259,-87.772934,"(41.951258958, -87.772934481)",2015,4,1,9,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,9995418,HY186092,2015-03-15 23:07:00,059XX S CAMPBELL AVE,041A,BATTERY,AGGRAVATED: HANDGUN,RESIDENCE PORCH/HALLWAY,False,False,...,2015,02/10/2018 03:50:01 PM,41.785660,-87.686257,"(41.785659563, -87.686257097)",2015,3,15,23,Sunday
29996,9998420,HY188618,2015-03-17 18:00:00,068XX S EAST END AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,2015,02/10/2018 03:50:01 PM,41.770648,-87.583787,"(41.770648384, -87.583786515)",2015,3,17,18,Tuesday
29997,9998516,HY188592,2014-11-19 10:14:00,011XX W DIVISION ST,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,OTHER,False,True,...,2014,02/10/2018 03:50:01 PM,41.903527,-87.656041,"(41.903527055, -87.656040878)",2014,11,19,10,Wednesday
29998,9999014,HY189066,2015-03-17 19:00:00,003XX E WACKER DR,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,False,...,2015,02/10/2018 03:50:01 PM,41.887804,-87.618633,"(41.887803629, -87.618633342)",2015,3,17,19,Tuesday


Sur 30 000 lignes de départ, il ne nous reste plus que 27 109. Soit une suppression de 2891 lignes contenant des valeurs manquantes.

In [11]:
# Verification à nouveau
data.isna().sum()

ID                      0
Case Number             0
Date                    0
Block                   0
IUCR                    0
Primary Type            0
Description             0
Location Description    0
Arrest                  0
Domestic                0
Beat                    0
District                0
Ward                    0
Community Area          0
FBI Code                0
X Coordinate            0
Y Coordinate            0
Year                    0
Updated On              0
Latitude                0
Longitude               0
Location                0
year                    0
month                   0
day                     0
hour                    0
weekday                 0
dtype: int64

Plus aucune valeur manquante. Nous poursuivons alors notre analyse.